In [1]:
%pip install -q -U kaggle_environments

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from kaggle_environments import make, evaluate

g:\PythonProject\RockPaperScissors\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


termcolor not installed, skipping dependency
No pygame installed, ignoring import
Loading environment lux_ai_s3 failed: No module named 'chex'


In [6]:
%%writefile rock_agent.py

#Агент, который всегда выбирает камень
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


In [7]:
%%writefile paper_agent.py

#Агент, который всегда выбирает бумагу
def your_agent(observation, configuration):
    return 1

Overwriting paper_agent.py


In [8]:
%%writefile scissors_agent.py

#Агент, который всегда выбирает ножницы
def your_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


In [9]:
%%writefile copy_opponent.py

import random

#Агент, который на первом ходу выбирает случайно камень, ножницы или бумагу, а дальше повторяет последнее действие опонента
def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


In [10]:
%%writefile random_agent.py


import random

# Агент, который всегда выбирает случайное действие (камень, ножницы или бумагу)
def random_agent(observation, configuration):
    return random.randrange(0, configuration.signs)

Overwriting random_agent.py


In [11]:
%%writefile crescendo_agent.py

strategy = [1, 2, 0]
# Агент, который последовательно выбирает бумагу, ножницы или камень
def paper_doll_agent(observation, configuration):
    global strategy
    if strategy:
        return strategy.pop(0)
    else:
        strategy = [1, 2, 0]
        return strategy.pop(0)

Overwriting crescendo_agent.py


In [12]:
%%writefile smart_agent.py

# Агент, который подстраивается под опонента. Первый ход агент всегда делает ножницами
# Если опонент выкинул в прошлый раз камень, значит агент выберет бумагу и т.д.
def smart_agent(observation, configuration):
    if observation.step > 0:
        return 2
    else:
        if observation.lastOpponentAction == 0:
            return 1
        elif observation.lastOpponentAction == 1:
            return 2
        else:
            return 0

Overwriting smart_agent.py


In [13]:
%%writefile three_random_two_deliberately.py

import random


count_random = 0
count_deliberately = 0
# Агент, который 3 раза выбирает случайный вариант, а потом 2 раза выбирает 
# выиграшный вариант относительно прошлого хода опонента
def three_random_two_deliberately(observation, configuration):
    global count_random, count_deliberately

    if count_random < 3:
        count_random += 1
        return random.randrange(0, configuration.signs)
    else:
        count_deliberately += 1
        
        if count_deliberately < 2:
            count_random = 0
            count_deliberately = 0

        if observation.lastOpponentAction == 0:
            return 1
        elif observation.lastOpponentAction == 1:
            return 2
        else:
            return 0

Overwriting three_random_two_deliberately.py


In [14]:
%%writefile losing_move_agent.py

# Агент, который также подстраивается под опонента, но он выбирает проигрышный вариант, относительно прошлого хода опонента
# Первый ход агент всегда делает бумагой. 
# Если опонент выкинул в прошлый раз камень, значит агент выберет ножницы и т.д. Такая стратегия поможет, если опонент никогда не повторяет
# свой прошлый ход, в таком случае у нас всегда будет или ничья или победа
def losing_move_agent(observation, configuration):
    if observation.step > 0:
        return 1
    else:
        if observation.lastOpponentAction == 0:
            return 2
        elif observation.lastOpponentAction == 1:
            return 0
        else:
            return 1

Overwriting losing_move_agent.py


In [15]:
%%writefile stack_of_money_agent.py


strategy = [0, 1, 1]
# Агент, который действует по стратегии: камень, бумага, бумага
def stack_of_money_agent(observation, configuration):
    global strategy
    if strategy:
        return strategy.pop(0)
    else:
        strategy = [0, 1, 1]
        return strategy.pop(0)

Overwriting stack_of_money_agent.py


In [16]:
%%writefile paper_doll_agent.py


strategy = [2, 1, 1]
# Агент, который действует по стратегии: ножницы, бумага, бумага
def paper_doll_agent(observation, configuration):
    global strategy
    if strategy:
        return strategy.pop(0)
    else:
        strategy = [2, 1, 1]
        return strategy.pop(0)

Overwriting paper_doll_agent.py


In [17]:
%%writefile sandwitch_agent.py


strategy = [1, 2, 1]
# Агент, который действует по стратегии: бумага, ножницы, бумага
def sandwitch_agent(observation, configuration):
    global strategy
    if strategy:
        return strategy.pop(0)
    else:
        strategy = [1, 2, 1]
        return strategy.pop(0)

Overwriting sandwitch_agent.py


In [18]:
%%writefile random_deliberately.py

import random


# Агент, который 1 раз выбирает случайный вариант, а потом выбирает выиграшный вариант относительно прошлого хода опонента
def random_deliberately(observation, configuration):
    if observation.step % 2 == 0:
        return random.randrange(0, configuration.signs)
    else:
        if observation.lastOpponentAction == 0:
            return 1
        elif observation.lastOpponentAction == 1:
            return 2
        else:
            return 0

Overwriting random_deliberately.py


In [5]:
evaluate(
    "rps", #environment to use - no need to change
    ["scissors_agent.py", "paper_agent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[99.0, -99.0]]

In [33]:
from typing import List


def tournament(teams: List) -> str:
    """
    Функция имитации турнира между созданными агентами
    teams List: список всех команд
    """

    dict_of_teams_and_victores = {i: 0 for i in teams}
    
    def match(agent1: str, agent2: str) -> str:
        """
        Функция обрабатывающая матч между 2 агентами
        agent1 str: первый агент, который принимает участие в матче. Передаётся в строковом формате
        agent2 str: второй агент, который принимает участие в матче. Передаётся в строковом формате
        """

        print('Матч между', agent1, 'vs', agent2)
        res = evaluate(
            "rps",
            [agent1, agent2],
            configuration={"episodeSteps": 100}
        )

        #Проверка, кто победил в матче.
        #Если у одного из агентов получилось None, автоматически побеждает его опонент
        #Если у опонентов общий счет [0, 0], автоматически побеждает первый агент
        #В остальных случаях идет простое сравнение счета, у кого счет по партиям больше, тот и победил
        if res[0][0] is None:
            print('Победитель матча', agent2)
            return agent2
        elif res[0][1] is None:
            print('Победитель матча', agent1)
            return agent1
        elif res[0][0] > res[0][1]:
            print('Победитель матча', agent1)
            return agent1
        elif res[0][0] < res[0][1]:
            print('Победитель матча', agent2)
            return agent2
        else:
            print('Победитель матча', agent1)
            return agent1
    
    #Цикл, который проходиться по всем агентам, чтобы все сыграли со всеми
    #Количество итераций цикла подчиняется формуле сочетания: C_n^k = n!/(k!*(n-k)!)
    #В нашем случае при 13 агентах: 
    # C_13^11 = 13! / (11! * (13 - 11)!) = 13! / (11! * 2!) = (13 * 12) / 2 = 13 * 6 = 78
    #Для уменьшения количество итераций и ускорения работы коды, желательно использовать турнирную сетку,
    #но по заданию сказано "запустить турнир между ними и проверить, какая стратегия показывает себя лучше всех.",
    #соответственно нам необходимо сыграть со всеми чтобы выяснить, кто лучше всего себя показывает.
    #При использовании турнирной сетки, агенту могут попасться соперники, против которых он показывает себя лучше, чем
    #против других. И тогда мы не сможем судить об эффективности той или иной стратегии
    for agent1 in range(len(teams)):
        if teams[agent1 + 1:]:
            print(f'Раунд {agent1 + 1}')
            for agent2 in teams[agent1 + 1:]:
                res = match(teams[agent1], agent2)
                dict_of_teams_and_victores[res] += 1
        print()

    winner = max(dict_of_teams_and_victores, key=dict_of_teams_and_victores.get)
    return f'Победитель турнира {winner}'


teams = [
    'copy_opponent.py', 
    'crescendo_agent.py', 
    'losing_move_agent.py', 
    'paper_agent.py', 
    'paper_doll_agent.py', 
    'random_agent.py',
    'random_deliberately.py',
    'rock_agent.py',
    'sandwitch_agent.py',
    'scissors_agent.py',
    'smart_agent.py',
    'stack_of_money_agent.py',
    'three_random_two_deliberately.py',
]

print(tournament(teams))

Раунд 1
Матч между copy_opponent.py vs crescendo_agent.py
Победитель матча crescendo_agent.py
Матч между copy_opponent.py vs losing_move_agent.py
Победитель матча copy_opponent.py
Матч между copy_opponent.py vs paper_agent.py
Победитель матча copy_opponent.py
Матч между copy_opponent.py vs paper_doll_agent.py
Победитель матча copy_opponent.py
Матч между copy_opponent.py vs random_agent.py
Победитель матча copy_opponent.py
Матч между copy_opponent.py vs random_deliberately.py
Победитель матча copy_opponent.py
Матч между copy_opponent.py vs rock_agent.py
Победитель матча copy_opponent.py
Матч между copy_opponent.py vs sandwitch_agent.py
Победитель матча copy_opponent.py
Матч между copy_opponent.py vs scissors_agent.py
Победитель матча copy_opponent.py
Матч между copy_opponent.py vs smart_agent.py
Победитель матча copy_opponent.py
Матч между copy_opponent.py vs stack_of_money_agent.py
Победитель матча copy_opponent.py
Матч между copy_opponent.py vs three_random_two_deliberately.py
Победит